In [1]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

#model loading
model = YOLO("YOLO11l.pt")

In [2]:
class_list = model.names
class_list

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [3]:
cap = cv2.VideoCapture('4.mp4')



In [5]:
line_y_red = 430

class_counts = defaultdict(int)

crossed_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist = True, classes = [1,2,3,5,6,7])
    #print(results)

    # filling result 
    if results and results[0].boxes is not None and results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        track_id = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidence = results[0].boxes.conf.cpu()

        cv2.line(frame, (690, line_y_red), (1130, line_y_red), (0, 0, 255), 3)
        #cv2.putText(frame, 'Red Line', (690, line_y_red - 10),
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)



           #loop through each detected obj
        for box, tid, class_idx, conf in zip(boxes, track_id, class_indices, confidence):
            x1, y1, x2, y2 = map(int, box)

            class_name = class_list[class_idx]

            cv2.putText(frame, f"ID: {tid} {class_name}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame,(x1, y1), (x2, y2), (0,255,0), 2)

            # Calculate center of the box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            # Draw a small dot (circle) at the center
            cv2.circle(frame, (center_x, center_y), radius=4, color=(0, 0, 255), thickness=-1)

            if center_y > line_y_red and tid not in crossed_ids:
                crossed_ids.add(tid)
                class_counts[class_name] += 1

        y_offset = 30
        for class_name, count in class_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            y_offset += 30



    cv2.imshow("YOLO object Tracking & Counting", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 258.5ms
Speed: 34.7ms preprocess, 258.5ms inference, 10.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 215.8ms
Speed: 1.9ms preprocess, 215.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 188.0ms
Speed: 1.0ms preprocess, 188.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 motorcycles, 3 buss, 1 truck, 192.4ms
Speed: 1.8ms preprocess, 192.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 motorcycles, 3 buss, 1 truck, 192.2ms
Speed: 1.1ms preprocess, 192.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 4 motorcycles, 3 buss, 2 trucks, 180.5ms
Speed: 1.0ms preprocess, 180.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 4 motorcycles, 3 buss, 2 